In [8]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [9]:
data = pd.read_csv('scdb-20rfq-15cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [10]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

1538

In [11]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

create_count = 0
create_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'CREATE':
        create_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        create_count += 1   


print(f"#CREATEs =  {create_count}")

#CREATEs =  499


In [12]:
df_test = grouped_data.get_group("a4e80f6dde276ba127239861a82fa460d251a25c149ea709498402b04a2fa5ad")

KeyError: 'a4e80f6dde276ba127239861a82fa460d251a25c149ea709498402b04a2fa5ad'

In [13]:
df_test

NameError: name 'df_test' is not defined

In [14]:
time_all_committed_creates = {}
time_passed_creates = []
validated_time = []
accepted_time = []
committed_time = []

create_underfit = []
create_underfit_dict = {}

create_overfit = []
create_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "CREATE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_creates[tx_id[i]] = actual_time
            time_passed_creates.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            create_underfit.append(temp_list)
            create_underfit_dict[tx_id(i)] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_creates[tx_id[i]] = actual_time
            time_passed_creates.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            create_overfit.append(temp_list)
            create_overfit_dict[tx_id[i]] = temp_list

In [15]:
# Analysis of CREATE Transaction
create_mean = "{:.2f}".format(stat.mean(time_passed_creates))
create_median = stat.median(time_passed_creates)
create_max = max(time_passed_creates)
create_min = min(time_passed_creates)
create_std_dev = "{:.2f}".format(stat.stdev(time_passed_creates))

print(create_mean)
print(create_median)
print(create_max)
print(create_min)
print(create_std_dev)
print(f"total creates = {len(time_passed_creates)}")

create_dict = {
               'min': create_min,
               'max': create_max,
               'average': create_mean,
               'median' : create_median,
               'standard_deviation' : create_std_dev,
               '# of txs' : len(time_passed_creates)}

80.32
67
384
62
24.94
total creates = 499


In [16]:
results = pd.DataFrame(create_dict, index = ['CREATE'])
results.to_csv('create_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
CREATE,62,384,80.32,67,24.94,499
